# Real Time Parametrized Aeroelastic Problem
## Author: Oriol CHANDRE VILA
## Tutors: Sylvain DUBREUIL (ONERA) et Joseph MORLIER (ISAE-SUPAERO)
### Internship in ONERA, within the Department of Systems and Information Treatment (DTIS).

The title of the internship is _Separated variables representation for the exploration of the Parametrized solution_. The methodology applied is based in an Offline-Online process, using an approach Greedy+POD for the phase offline and a Kriging interpolation in order to solve really fast cases in the Online phase (or real time phase).

This methodology has been applied to ONERA's code __aero_struct__ which computes the aeroelastic problem for a aircraft wing, type Airbus.

## I- OFFLINE PHASE
This program aims to use the POD-ROM to create a reduced basis able to be used in a real-time application.
It is applied to an aeroelastic problem. A Greedy algorithm has been used to introduced a POD based on 
the snapshots method combined with the Singular Value Decomposition (SVD).
Using version 3.0.6 of GMSH.

In [ ]:
import numpy as np
from scipy import linalg
from pyDOE import lhs
import functions_Offline as foff
import timeit
from sklearn.externals import joblib
from gaussian_process import GaussianProcess

### PARAMETERS BOUNDS: h_skins, h_ribs, h_spars_le, h_spars_te, b, S
h_* = thickness of different parts (m)

b = wing span (m)

S = wing surface (m^2)

- This parameters are the ones chosen to vary in our problem.

In [ ]:
pMin_i = np.array([0.028,0.009,0.017,0.009,64.75,443.56])
pMax_i = np.array([0.031,0.015,0.025,0.015,75.65,500.21])
pMin = np.zeros((6,1))
pMin[:,0]= pMin_i[:]
pMax = np.zeros((6,1))
pMax[:,0]= pMax_i[:]
np.save("../results/Offline/pMin",pMin)
np.save("../results/Offline/pMax",pMax)

### OTHERS PARAMETERS OF INTEREST
alpha = angle of attack (°)

M_inf = Mach number

E = Young modulus (Pa)

nu = Poisson's ratio

BF = distance of the fuselage over the wing (m)

phi_d = sweep angle (°)

diedre_d = dihedral angle (°)

n_ribs_1 = number of ribs (first sector)

n_ribs_2 = number of ribs (second sector)

- This parameters are needed to define the geometry and material of the wing

In [ ]:
alpha = 2.5 
M_inf = 0.85
E = 70.*1e9
nu = 0.3
BF = 5.96
phi_d = 30.
diedre_d = 1.5
n_ribs_1 =  8
n_ribs_2 =  12

## FIXED PARAMETERS (Flight Altitude = 11000 meters)
rho = density at 11000 meters (kg/m^3)

Pressure = pressure at 11000 meters (Pa)

adiabatic_i = adiabatic index of air

a_speed = speed of sound at 11000 meters (m/s)

phi = sweep angle (rad)

diedre = dihedral angle (rad)

In [ ]:
rho = 0.3629
Pressure = 22552.
adiabiatic_i = 1.4
a_speed = np.sqrt(adiabiatic_i*Pressure/rho)
v_inf = M_inf*a_speed
phi = phi_d*np.pi/180.
diedre = diedre_d*np.pi/180.

Creation of the mesh files: VLM and FEM

In [ ]:
foff.create_mesh_files(n_ribs_1,n_ribs_2)

Initialisation of the problem

In [ ]:
param_data = np.zeros(9)
param_data[0] = alpha
param_data[1] = v_inf
param_data[2] = rho
param_data[3] = E
param_data[4] = nu

### GREEDY ALGORITHM

In [ ]:
tic = timeit.default_timer()

# 0) Initialization
nSamples = 3
nIterations = nSamples-1
nCandidates = 5
nParam = len(pMin)
pCandidate = np.dot(pMin,np.ones((1,nCandidates))) + lhs(nParam,nCandidates).T*np.dot((pMax-pMin),np.ones((1,nCandidates)))
indexCand = np.zeros((nCandidates,), dtype=int)
for i in range(nCandidates):
    indexCand[i] = i
pCandMax = np.zeros((nParam,nSamples))

# 1) Ramdomly select a first sample (in the middle of the domain)
pMiddle = (pMin+pMax)/2
param_data[5] = pMiddle[0,0]
param_data[6] = pMiddle[1,0]
param_data[7] = pMiddle[2,0]
param_data[8] = pMiddle[3,0]
b = pMiddle[4,0]
S = pMiddle[5,0]
pCandMax[:,0] = pMiddle[:,0]
# Calculating the number of nodes and initialization of samples matrices
n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(param_data)
uSamples = np.zeros((n_FEM_nodes*6,nSamples)) # POD displacement
gSamples = np.zeros((n_gamma_nodes,nSamples)) # POD circulation

# 2) Solving HDM-based problem
print "Greedy Iteration num.: 0\n",
uMiddle,gMiddle = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
uSamples[:,0] = uMiddle
gSamples[:,0] = gMiddle

# 3) Building the ROB
uV,_,_ = linalg.svd(uSamples,full_matrices=False)
gV,_,_ = linalg.svd(gSamples,full_matrices=False)
uV_tr = np.transpose(uV)
gV_tr = np.transpose(gV)

# 4) Iteration loop
for iIteration in range(nIterations):
    print "Greedy Iteration num.: ",iIteration+1
    
    # 5) Solve argmax(residual)
    maxError = 0.
    candidateMaxError = 0
    for iCandidate in indexCand:
        # 5a) Loading candidates information
        param_data[5] = pCandidate[0,iCandidate]
        param_data[6] = pCandidate[1,iCandidate]
        param_data[7] = pCandidate[2,iCandidate]
        param_data[8] = pCandidate[3,iCandidate]
        b = pCandidate[4,iCandidate]
        S = pCandidate[5,iCandidate]
        A, B = foff.VLM_params(param_data,b,S,phi,diedre,BF,Mach=M_inf)
        Ar = np.dot(np.dot(gV_tr,A),gV)
        Br = np.dot(gV_tr,B)
        Ar_lu = linalg.lu_factor(Ar)
        gr = linalg.lu_solve(Ar_lu,-Br)
        g_exp = np.dot(gV,gr)
        F_s = foff.get_Fs(g_exp,param_data)
        K, Fs = foff.get_FEM(param_data,F_s)
        Kr = np.dot(np.dot(uV_tr,K),uV)
        Fr = np.dot(uV_tr,Fs)
        q = linalg.solve(Kr,Fr)
        # 5b) Compute error indicator
        Y1 = np.dot(K,np.dot(uV,q))- Fs
        errorIndicator = np.dot(np.transpose(Y1),Y1)
        if (iCandidate == 0 or maxError < errorIndicator):
            candidateMaxError = iCandidate
            maxError = errorIndicator
            
    # 6) Solving HDM-based problem for the candidate with the highest error indicator
    param_data[5] = pCandidate[0,candidateMaxError]
    param_data[6] = pCandidate[1,candidateMaxError]
    param_data[7] = pCandidate[2,candidateMaxError]
    param_data[8] = pCandidate[3,candidateMaxError]
    b = pCandidate[4,candidateMaxError]
    S = pCandidate[5,candidateMaxError]
    pCandMax[:,iIteration+1] = pCandidate[:,candidateMaxError]
    uIter,gIter = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
    uSamples[:,iIteration+1] = uIter
    gSamples[:,iIteration+1] = gIter
    
    # 7) Building the ROB
    uV,_,_ = linalg.svd(uSamples[:,0:iIteration+2],full_matrices=False)
    gV,_,_ = linalg.svd(gSamples[:,0:iIteration+2],full_matrices=False)
    uV_tr = np.transpose(uV)
    gV_tr = np.transpose(gV)
    ##UPDATING indexCand: The current index must be removed!
    xC = 0
    while True:
        if indexCand[xC] == candidateMaxError:
            break
        xC += 1
    indexCand = np.delete(indexCand,xC)

toc = timeit.default_timer()
print("TOTAL COMPUTATION TIME: "+str((toc-tic)/60.)+" min")

### KRIGING

In [ ]:
# Initialization
tic = timeit.default_timer()
U_comp = uSamples
G_comp = gSamples

# Weight calculations
a_kri = np.zeros((nSamples,nSamples))
b_kri = np.zeros((nSamples,nSamples))
for i in range(nSamples):
    for j in range(nSamples):
        a_kri[j,i] = np.dot(U_comp[:,j].T,uV[:,i])
        b_kri[j,i] = np.dot(G_comp[:,j].T,gV[:,i])

# Trained solution
mean = "constant"
covariance = "squared_exponential"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
for i in range(nSamples):
    GP_u = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_u.fit(pCandMax.T, a_kri[:,i])
    GP_g = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_g.fit(pCandMax.T, b_kri[:,i])
    joblib.dump(GP_u, "../results/Offline/GP_alpha_"+str(i)+".pkl")
    joblib.dump(GP_g, "../results/Offline/GP_beta_"+str(i)+".pkl")
    
toc = timeit.default_timer()
print("KRIGING COMPUTATION TIME: "+str(toc-tic)+" s")

## II- ONLINE PHASE
Using the data saved within the two Kriging structures, real-time solutions can be computed in a really fast way.

In [ ]:
from mesh_deformation_airbus import mesh_deformation_airbus
from VLM import VLM_study
from FEM import FEM_study
import subprocess

From the Offline Phase we will use the parameters: _nSamples_, _param data_, _uV_ and _gV_. Besides, the data of the Kriging structures.

### Definition of the case
Here, the user choose the values of his analysis!

In [ ]:
h_skins = 0.028
h_ribs = 0.009
h_spars_le = 0.017
h_spars_te = 0.009
b = 64.75
S = 443.56

Initialisation 

In [ ]:
tic = timeit.default_timer()
x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
pC = np.zeros((6))
pC[0] = param_data[5] = h_skins
pC[1] = param_data[6] = h_ribs
pC[2] = param_data[7] = h_spars_le
pC[3] = param_data[8] = h_spars_te
pC[4] = b
pC[5] = S

1) Kriging prediction of the interested point

In [ ]:
u_mp = np.zeros((1,nSamples))
u_vp = np.zeros((1,nSamples))
gamma_mp = np.zeros((1,nSamples))
gamma_vp = np.zeros((1,nSamples))
for i in range(nSamples):
    # *Loading Kriging data
    name_a = joblib.load("../results/Offline/GP_alpha_"+str(i)+".pkl")
    name_b = joblib.load("../results/Offline/GP_beta_"+str(i)+".pkl")
    # **Prediction
    u_mp[0,i], u_vp[0,i] = name_a.predict(x_conf.reshape((1,6)), eval_MSE=True)
    gamma_mp[0,i], gamma_vp[0,i] = name_b.predict(x_conf.reshape((1,6)), eval_MSE=True)
u_mean = 0
u_var2 = 0
gamma_mean = 0
gamma_var2 = 0
for i in range(nSamples):
    u_mean += np.dot(u_mp[0,i],uV[:,i])
    u_var2 += np.dot(u_vp[0,i]**2,uV[:,i]**2)
    gamma_mean += np.dot(gamma_mp[0,i],gV[:,i])
    gamma_var2 += np.dot(gamma_vp[0,i]**2,gV[:,i]**2)

2) Calculation of u_est and g_est

In [ ]:
u_est = u_mean
u_est1 = u_mean+3.0*np.sqrt(u_var2)
u_est2 = u_mean-3.0*np.sqrt(u_var2)
g_est = gamma_mean
g_est1 = gamma_mean+3.0*np.sqrt(gamma_var2)
g_est2 = gamma_mean-3.0*np.sqrt(gamma_var2)
toc = timeit.default_timer()
print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")

3) Publication of the results: u_est, g_est

In [ ]:
a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
a_new.new_wing_mesh(b,S)
vlm_mesh = '../mesh/param_wing/new_VLM.msh'
fem_mesh = '../mesh/param_wing/new_FEM.msh'
n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)

# 3a) Computation of strains and stress
element_property,material,element_type = foff.create_dict(param_data)
my_fem = FEM_study(fem_mesh,element_type,element_property,material)
my_fem.read_mesh_file()
strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)

# 3b) u_est
my_fem.post_processing(u_est,"../results/Param_wing/u_est")
my_fem.post_processing_var1(u_est1,"../results/Param_wing/u_est1")
my_fem.post_processing_var2(u_est2,"../results/Param_wing/u_est2")

## 3c) Von Mises stress
Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
my_fem.post_processing_Von_Mises(Von_Mises_Stress,'../results/param_wing/result_VM_iter')
print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")
stress_max = 0.
stress_max_i = 0
for i in range(len(Von_Mises_Stress)):
    stress_i = Von_Mises_Stress[i]
    if (stress_max < stress_i):
        stress_max = stress_i
        stress_max_i = i
print("Node VM max = "+str(stress_max_i))
print("Von Mises Stress max = "+str(Von_Mises_Stress[stress_max_i])+" Pa")

# 3d) g_est
my_vlm = VLM_study(vlm_mesh)
my_vlm.post_processing_gamma('Param_wing/g_est',g_est)
my_vlm.post_processing_gamma_var1('Param_wing/g_est1',g_est1)
my_vlm.post_processing_gamma_var2('Param_wing/g_est2',g_est2)

4) Quality of the prediction

In [ ]:
for x in range(len(u_mean)):
    if u_mean[x] == 0:
        u_mean[x] = 1.*1e-35
tau_u = abs(np.sqrt(u_var2)/u_mean)
tau_g = abs(np.sqrt(gamma_var2)/gamma_mean)
my_fem.post_processing_erreur(tau_u,"../results/Param_wing/tau_u")
my_vlm.post_processing_erreur('Param_wing/tau_g',tau_g)

5) Opening GMSH files

In [ ]:
subprocess.call(['D://gmsh-3.0.6/gmsh','../results/Param_wing/g_est.msh','../results/Param_wing/g_est1.msh','../results/Param_wing/g_est2.msh','../results/Param_wing/tau_g.msh','-open'])
subprocess.call(['D://gmsh-3.0.6/gmsh','../results/Param_wing/u_est.msh','../results/Param_wing/u_est1.msh','../results/Param_wing/u_est2.msh','../results/Param_wing/tau_u.msh','-open'])
subprocess.call(['D://gmsh-3.0.6/gmsh','../results/Param_wing/result_VM_iter.msh','-open'])